In [2]:
import time
import json
import os
import pickle
current_dir=os.getcwd()
# print(current_dir)
os.chdir("../Paulihedral_v2/Paulihedral_new")
# from parallel_bl import depth_oriented_scheduling
from benchmark.mypauli import pauliString
from arch.__init__ import graph_from_coupling
import synthesis_FT, parallel_bl, synthesis_SC
import importlib
importlib.reload(synthesis_FT)
importlib.reload(parallel_bl)
os.chdir(current_dir)
from copy import deepcopy
from qiskit import transpile
from qiskit.transpiler import CouplingMap


c:\Users\main\miniconda3\envs\qiskit_ph\lib\site-packages\qiskit\__init__.py:75: RuntimeWarning: Could not import the IBMQ provider from the qiskit-ibmq-provider package. Install qiskit-ibmq-provider or check your installation.
  RuntimeWarning)


In [3]:
def paulihedral(parr):
    # nq = len(parr[0][0])
    # length = nq//2 # `length' is a hyperparameter, and can be adjusted for best performance
    # a1 = parallel_bl.depth_oriented_scheduling(parr, length=length, maxiter=30)
    print(parr)
    lnq = len(parr[0][0])
    length = lnq // 2 # `length' is a hyperparameter, and can be adjusted for best performance. Here we keep `length' fixed for simplicity.
    coup = CouplingMap().from_grid(11,11)
    edges=[e for e in coup.graph.edges]
    graph=graph_from_coupling(edges)
    # a2 = parallel_bl.depth_oriented_scheduling(parr, length=length, maxiter=30)
    a1 = parallel_bl.gate_count_oriented_scheduling(parr)
    # print("a1: ", a1)
    # a1=[[[p]] for block1 in a1 for block2 in block1 for p in block2] #flatten
    print("a1: ", a1)
    qc = synthesis_SC.block_opt_SC(a1, graph=graph, arch='manhattan') # The arch parameter won't do anything if you follow the code.

    # qc = synthesis_FT.block_opt_FT(a1)
    # print(qc)

    return qc

# def paulihedral(parr):
#     # nq = len(parr[0][0])
#     # length = nq//2 # `length' is a hyperparameter, and can be adjusted for best performance
#     # a1 = parallel_bl.depth_oriented_scheduling(parr, length=length, maxiter=30)
#     print(parr)
#     a1 = parallel_bl.gate_count_oriented_scheduling(parr)
#     print("a1: ", a1)
#     a1=[[[p]] for block1 in a1 for block2 in block1 for p in block2] #flatten
#     print("a1: ", a1)
#     qc = synthesis_FT.block_opt_FT(a1)
#     # print(qc)

#     return qc

def is_all_identity(pauli):
    return all(char == 'I' for char in pauli)

# def update_json_file(filename, runtime, cx_count, circ_depth):
#     with open(f'benchmarks/results/test_' + filename, 'r') as paulis_file:
#         data=json.load(paulis_file)
#     print(data[1])
#     # with open(f'benchmarks/results/test_' + filename, 'w') as paulis_file:
#     #     json.dump([test_paulis, results], paulis_file, indent=4)

In [4]:
#Compare json files in a specific folder
def run_experiment_folder(folder_path = None, filename = None, save_output = False):

    if filename == None:
        file_list = os.listdir(folder_path)
    else:
        file_list = [filename]
    # Iterate over all files in the folder
    for filename in file_list:
        # Check if the file is a JSON file
        if filename.endswith(".json"):
            results = []
            # Print the filename
            print(filename)
            with open(folder_path + '/' + filename, "r") as file:
                test_paulis = json.load(file)
                origin_paulis=deepcopy(test_paulis) # we will save this.
            if filename.startswith(("max_cut", "labs")):
                test_paulis = test_paulis[0] # only get the cost hamiltonian
                print(test_paulis)
            number_of_ham=len(test_paulis)
            test_params = [0.01 * i for i in range(len(test_paulis))]

            # Measure time for method
            start_time = time.time()
            # Filter the list to remove all identity Paulis
            paulis = [[pauliString(pauli, 1.0)] for pauli in test_paulis if not is_all_identity(pauli)]
            circuit = paulihedral(paulis)
            # circuit = transpile(circuit, basis_gates=["cx", "sx", "x", "rz"], optimization_level=3)
            end_time = time.time()
            tot_time = end_time - start_time
        
            # Collect results
            result = {
                "num_paulis": number_of_ham,
                "times": {
                    "paulihedral_time": tot_time
                },
                # "gate_counts": {
                #     "paulihedral_method": circuit.count_ops().get('cx', 0)
                # },
                # "circuit_depth": {
                #     "paulihedral_method": circuit.depth()
                # },
                # "test_paulis_file": f'benchmarks/results/paulihedral_partial_results/' + filename
            }
            print(result)
            results.append(result)
            if save_output == True:
                # Save to pickle files
                save_filename=filename[0:-len(".json")]+".pkl"
                with open(f'benchmarks/paulihedral_partial_results_topology_ibm_grid/' + save_filename, 'wb') as paulis_file:
                    pickle.dump({"paulis": origin_paulis, "circuit": circuit.qasm(), "results": results}, paulis_file)
                # with open(f'benchmarks/paulihedral_partial_results/' + save_filename, 'rb') as paulis_file:
                #     print("printing data: ", pickle.load(paulis_file))
    

    

In [5]:
#Compare a given list of paulis
def run_experiment_paulis(test_paulis, test_params = None, save_output = False, filename = "Paulis"):

    results = []
    # paulis = test_paulis
    # Filter the list to remove all identity Paulis
    if test_params is None:
        test_params = [0.01 * i for i in range(len(test_paulis))]
    
    number_of_ham=len([p for block in test_paulis for p in block])
    # Measure time for method
    start_time = time.time()
    paulis = [[pauliString(p, 1.0) for p in block] for block in test_paulis]
    # paulis = [[pauliString(p, 1.0)] for p in test_paulis]
    circuit= paulihedral(paulis)
    # circuit = transpile(circuit, basis_gates=["cx", "sx", "x", "rz"], optimization_level=3)
    # print(circuit)
    end_time = time.time()
    tot_time = end_time - start_time

    # Collect results
    result = {
        "num_paulis": number_of_ham,
        "times": {
            "paulihedral_time": tot_time
        }
        # "gate_counts": {
        #     "paulihedral_method": circuit.count_ops().get('cx', 0)
        # },
        # "circuit_depth": {
        #     "paulihedral_method": circuit.depth()
        # },
        # "test_paulis_file": f'benchmarks/paulihedral_partial_results' + filename
    }
    print(result)
    results.append(result)
    if save_output == True:
        # Save to pickle files
        with open(f'benchmarks/paulihedral_partial_results_topology_ibm_grid/' + filename, 'wb') as paulis_file:
            pickle.dump({"paulis": test_paulis, "circuit": circuit.qasm(), "results": results}, paulis_file)
        # with open(f'benchmarks/paulihedral_partial_results/' + filename, 'rb') as paulis_file:
        #     print("printing data: ", pickle.load(paulis_file))
    return circuit
    

    

In [6]:
#first compare the UCCSD ansatz
electrons_list = [2, 2, 4, 6, 8, 10,]# 12]
orbitals_list = [4, 6, 8, 12, 16, 20]#, 24]
# paulis_len=[24, 80, 320, 1656, 5376, 13400]
#First evaluate the UCCSD ansatz:
for e, o in zip(electrons_list, orbitals_list):
    filename=f"uccsd_hamiltonian_e{e}_o{o}.json"
    with open(f"benchmarks/uccsd_hamiltonians/{filename}", "r") as file:
        data=json.load(file)
    print(f"initial data: {data}")
    num_qubits=len(data[0][0])
    
    # data=[[p for p in block if not is_all_identity(p)] for block in data]
    # data=[block for block in data if len(block)>0]
    # data=[p for block in data for p in block if not is_all_identity(p)] #flatten
    num_hams=len([p for block in data for p in block])
    print(data)
    entanglers = run_experiment_paulis(data, num_qubits, save_output = True, filename=f"Paulis{num_hams}.pkl")

initial data: [['YZXI', 'XZXI', 'YZYI', 'XZYI'], ['IYZX', 'IXZX', 'IYZY', 'IXZY'], ['YXYX', 'XXYX', 'YYYX', 'XYYX', 'YXYY', 'XXYY', 'YYYY', 'XYYY', 'YXXX', 'XXXX', 'YYXX', 'XYXX', 'YXXY', 'XXXY', 'YYXY', 'XYXY']]
[['YZXI', 'XZXI', 'YZYI', 'XZYI'], ['IYZX', 'IXZX', 'IYZY', 'IXZY'], ['YXYX', 'XXYX', 'YYYX', 'XYYX', 'YXYY', 'XXYY', 'YYYY', 'XYYY', 'YXXX', 'XXXX', 'YYXX', 'XYXX', 'YXXY', 'XXXY', 'YYXY', 'XYXY']]
[[YZXI, XZXI, YZYI, XZYI], [IYZX, IXZX, IYZY, IXZY], [YXYX, XXYX, YYYX, XYYX, YXYY, XXYY, YYYY, XYYY, YXXX, XXXX, YYXX, XYXX, YXXY, XXXY, YYXY, XYXY]]
a1:  [[[IYZX, IXZX, IYZY, IXZY]], [[YXYX, XXYX, YYYX, XYYX, YXYY, XXYY, YYYY, XYYY, YXXX, XXXX, YYXX, XYXX, YXXY, XXXY, YYXY, XYXY]], [[YZXI, XZXI, YZYI, XZYI]]]
{'num_paulis': 24, 'times': {'paulihedral_time': 0.9253859519958496}}
initial data: [['YZXIII', 'XZXIII', 'YZYIII', 'XZYIII'], ['YZZZXI', 'XZZZXI', 'YZZZYI', 'XZZZYI'], ['IYZXII', 'IXZXII', 'IYZYII', 'IXZYII'], ['IYZZZX', 'IXZZZX', 'IYZZZY', 'IXZZZY'], ['YXYXII', 'XXYXII', '

In [7]:
#Then compare the Hamiltonian simulation paulis in HS_paulis folder
run_experiment_folder(folder_path = "benchmarks/HS_paulis", save_output = True)

benzene.json
[[IIIIIIIIIIIZ], [IIIIIIIIIIYY], [IIIIIIIIIIXX], [IIIIIIIIIYZY], [IIIIIIIIIXZX], [IIIIIIIIYZZY], [IIIIIIIIXZZX], [IIIIIIIYZZZY], [IIIIIIIXZZZX], [IIIIIIYZZZZY], [IIIIIIXZZZZX], [IIIIIIIIIIZI], [IIIIIIIIIYYI], [IIIIIIIIIXXI], [IIIIIIIIYZYI], [IIIIIIIIXZXI], [IIIIIIIYZZYI], [IIIIIIIXZZXI], [IIIIIIYZZZYI], [IIIIIIXZZZXI], [IIIIIIIIIZII], [IIIIIIIIYYII], [IIIIIIIIXXII], [IIIIIIIYZYII], [IIIIIIIXZXII], [IIIIIIYZZYII], [IIIIIIXZZXII], [IIIIIIIIZIII], [IIIIIIIYYIII], [IIIIIIIXXIII], [IIIIIIYZYIII], [IIIIIIXZXIII], [IIIIIIIZIIII], [IIIIIIYYIIII], [IIIIIIXXIIII], [IIIIIIZIIIII], [IIIIIZIIIIII], [IIIIYYIIIIII], [IIIIXXIIIIII], [IIIYZYIIIIII], [IIIXZXIIIIII], [IIYZZYIIIIII], [IIXZZXIIIIII], [IYZZZYIIIIII], [IXZZZXIIIIII], [YZZZZYIIIIII], [XZZZZXIIIIII], [IIIIZIIIIIII], [IIIYYIIIIIII], [IIIXXIIIIIII], [IIYZYIIIIIII], [IIXZXIIIIIII], [IYZZYIIIIIII], [IXZZXIIIIIII], [YZZZYIIIIIII], [XZZZXIIIIIII], [IIIZIIIIIIII], [IIYYIIIIIIII], [IIXXIIIIIIII], [IYZYIIIIIIII], [IXZXIIIIIIII], [YZZYIIIII

In [8]:
#Then compare the MAXCUT paulis in max_cut_paulis folder

run_experiment_folder(folder_path = "benchmarks/max_cut_paulis_blocks", save_output = True)

# all_filenames=["max_cut_benchmark_regular_n20_e4_l1.json", "max_cut_benchmark_regular_n20_e4_l3.json", 
#                "max_cut_benchmark_regular_n20_e8_l1.json", "max_cut_benchmark_regular_n20_e8_l3.json",
#                "max_cut_benchmark_regular_n20_e12_l1.json", "max_cut_benchmark_regular_n20_e12_l3.json"]
# for filename in all_filenames:
#     run_experiment_folder(folder_path = "benchmarks/max_cut_paulis_blocks", filename=filename, save_output = True)

max_cut_benchmark_connected_n10_e12_l1.json
['IIIIIIZIZI', 'IIZIIIIIIZ', 'IZIIIIIIZI', 'IIIIZIIIZI', 'ZZIIIIIIII', 'IIIIIZIIIZ', 'IIIZIIIIZI', 'IIIIZIIIIZ', 'IIZIIZIIII', 'IZIIIIIIIZ', 'IIIZIIIZII', 'IIIIIZIZII']
[[IIIIIIZIZI], [IIZIIIIIIZ], [IZIIIIIIZI], [IIIIZIIIZI], [ZZIIIIIIII], [IIIIIZIIIZ], [IIIZIIIIZI], [IIIIZIIIIZ], [IIZIIZIIII], [IZIIIIIIIZ], [IIIZIIIZII], [IIIIIZIZII]]
a1:  [[[IIIIIIZIZI]], [[IIIIIZIIIZ]], [[IIIIIZIZII]], [[IIIIZIIIIZ]], [[IIIIZIIIZI]], [[IIIZIIIIZI]], [[IIIZIIIZII]], [[IIZIIIIIIZ]], [[IIZIIZIIII]], [[IZIIIIIIIZ]], [[IZIIIIIIZI]], [[ZZIIIIIIII]]]
{'num_paulis': 12, 'times': {'paulihedral_time': 0.9036078453063965}}
max_cut_benchmark_connected_n10_e12_l3.json
['IIIIIIZIZI', 'IIZIIIIIIZ', 'IZIIIIIIZI', 'IIIIZIIIZI', 'ZZIIIIIIII', 'IIIIIZIIIZ', 'IIIZIIIIZI', 'IIIIZIIIIZ', 'IIZIIZIIII', 'IZIIIIIIIZ', 'IIIZIIIZII', 'IIIIIZIZII']
[[IIIIIIZIZI], [IIZIIIIIIZ], [IZIIIIIIZI], [IIIIZIIIZI], [ZZIIIIIIII], [IIIIIZIIIZ], [IIIZIIIIZI], [IIIIZIIIIZ], [IIZIIZIIII], [IZIIIIIII

In [9]:
# #Labs

run_experiment_folder(folder_path = "benchmarks/labs_paulis_blocks", save_output = True)

labs_n10_layers1.json
['IIZZZZIIII', 'ZZIIIIZZII', 'ZZIIZZIIII', 'IIZZIIZZII', 'ZZIIIIIIZZ', 'IIZZIZZIII', 'IIZZIIIIZZ', 'ZZIIIZZIII', 'ZZZZIIIIII', 'IIIIZZZZII', 'ZZIIIIIZZI', 'IIIZZZZIII', 'IIZZIIIZZI', 'IZIIIIIIIZ', 'IZIIIZIIII', 'ZIIIIIZIII', 'ZIIIZIIIII', 'IIIIIZIIIZ', 'ZIIIZZIIIZ', 'IZIIZIZIIZ', 'IIIIZIZIII', 'IIIIZZIZZI', 'ZIZIIIIIII', 'IZIZIIIIII', 'IIIIZZIIZZ', 'IZZIIIZZII', 'IIIIIIZZZZ', 'ZZIZZIIIII', 'IZZIIIIIZZ', 'IIIZZIZZII', 'IZZZZIIIII', 'IIIIIZZIZZ', 'IZZIZZIIII', 'IIIIIIIZIZ', 'IIIIIIZIZI', 'IZZIIZZIII', 'IIIZZIIIZZ', 'IZZIIIIZZI', 'IIIZIIIIIZ', 'ZIIZIIZIIZ', 'IIIZIZIIII', 'IIIZZIIZZI', 'IIZIIZZIIZ', 'IIIIIZZZZI', 'IIZIZIIIII', 'IZIZIIZIZI', 'ZIZIIZIZII', 'ZIZIIIZIZI', 'IIIZIIIZII', 'IZIIIIIZII', 'IIIZIZIZIZ', 'IZIZIIIZIZ', 'IZIZIZIZII', 'IIIIIZIZII', 'ZIZIZIZIII', 'IZIZZIZIII', 'ZIZIIIIZIZ', 'IIIIZIZZIZ', 'ZIZZIZIIII', 'IIZIZZIZII', 'ZIIIIIIIZI', 'IIZIZIIZIZ', 'IIIZIZZIZI', 'IIZIZIZIZI', 'ZIIZIZIIZI', 'IIZIIIZIII', 'IZIIZZIIZI', 'ZIIZZIIZII', 'IIZIIIIIZI', 'IIIIZIIIZI